<a href="https://colab.research.google.com/github/umar-faroooq/Jupyter-notebooks/blob/main/Autoencoders_on_giga_mall_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import tensorflow as tf
import os

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/LSTM Implementation/AHUF5/preprocessed_AHUF5.csv')
date_time = pd.to_datetime(df.pop('Timestamp'))

In [ ]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
hour,184556.0,11.481442,6.996583,0.00,5.000000,11.000000,18.000000,23.00
day,184556.0,2.983674,2.049272,0.00,1.000000,3.000000,5.000000,6.00
OAT,184556.0,29.578779,4.235406,19.97,26.432292,29.019776,32.603375,43.57
SAT,184556.0,21.532370,6.854144,0.00,18.100000,21.300000,27.100000,34.30
RH,184556.0,56.528214,18.727590,0.00,49.000000,59.000000,70.000000,91.00
CHW_Temp,184556.0,19.317503,7.275348,0.00,15.700000,18.800000,23.600000,34.00
Valve,184556.0,38.702481,48.552425,0.00,0.000000,0.000000,100.000000,100.00
RAT,184556.0,27.146891,6.456997,0.00,27.500000,28.300000,29.300000,35.70


In [ ]:
column_indices = {name: i for i, name in enumerate(df.columns)}

n = len(df)
train_df = df[0:int(n*0.7)]
val_df = df[int(n*0.7):int(n*0.9)]
test_df = df[int(n*0.9):]

num_features = df.shape[1]

In [ ]:
train_df

,hour,day,OAT,SAT,RH,CHW_Temp,Valve,RAT
0,0,6,25.790000,27.3,35,28.4,100,28.0
1,0,6,25.756667,27.3,35,28.4,100,28.0
2,0,6,25.723333,27.3,35,28.4,100,28.0
3,0,6,25.690000,27.3,35,28.4,100,28.0
4,0,6,25.656667,27.3,35,28.4,100,28.0
...,...,...,...,...,...,...,...,...
129184,23,5,28.104000,25.9,69,24.9,0,27.8
129185,23,5,28.101667,25.9,69,24.9,0,27.8
129186,23,5,28.099333,25.9,69,24.9,0,27.8
129187,23,5,28.097000,25.9,69,24.9,0,27.8


In [ ]:
val_df

,hour,day,OAT,SAT,RH,CHW_Temp,Valve,RAT
129189,23,5,28.092333,25.9,69,24.9,0,27.8
129190,0,6,28.090000,25.9,69,24.9,0,27.8
129191,0,6,28.073167,25.9,69,24.9,0,27.8
129192,0,6,28.056333,25.9,69,24.9,0,27.8
129193,0,6,28.039500,25.9,69,24.9,0,27.8
...,...,...,...,...,...,...,...,...
166095,6,1,26.494000,26.8,74,17.1,100,29.4
166096,6,1,26.512000,26.8,74,17.1,100,29.4
166097,7,1,26.530000,26.9,74,17.1,100,29.4
166098,7,1,26.535000,26.9,74,17.2,100,29.5


In [ ]:
column_indices = {name: i for i, name in enumerate(df.columns)}

n = len(df)
train_df = df[0:int(n*0.7)]
val_df = df[int(n*0.7):int(n*0.9)]
test_df = df[int(n*0.9):]

num_features = df.shape[1]

In [ ]:
train_mean = train_df.mean()
train_std = train_df.std()

train_df = (train_df - train_mean) / train_std
val_df = (val_df - train_mean) / train_std
test_df = (test_df - train_mean) / train_std

In [ ]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df=train_df, val_df=val_df, test_df=test_df,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

In [ ]:
def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window

In [ ]:
def make_dataset(self, data):
  data = np.array(data, dtype=np.float32)
  ds = tf.keras.preprocessing.timeseries_dataset_from_array(
      data=data,
      targets=None,
      sequence_length=self.total_window_size,
      sequence_stride=1,
      shuffle=True,
      batch_size=32,)

  ds = ds.map(self.split_window)

  return ds

WindowGenerator.make_dataset = make_dataset

In [ ]:
@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
    # And cache it for next time
    self._example = result
  return result

WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example

In [ ]:
OUT_STEPS = 60
multi_window = WindowGenerator(input_width=120,
                               label_width=OUT_STEPS,
                               shift=OUT_STEPS)


multi_window

Total window size: 180
Input indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]
Label indices: [120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155
 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173
 174 175 176 177 178 179]
Label column name(s): None

In [ ]:
MAX_EPOCHS = 20

def compile_and_fit(model, window, patience=2):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf.metrics.MeanAbsoluteError()])

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

In [ ]:
class FeedBack(tf.keras.Model):
  def __init__(self, units, out_steps):
    super().__init__()
    self.out_steps = out_steps
    self.units = units
    self.lstm_cell = tf.keras.layers.LSTMCell(units)
    # Also wrap the LSTMCell in an RNN to simplify the `warmup` method.
    self.lstm_rnn = tf.keras.layers.LSTM(self.lstm_cell, return_state=True)
    self.dense = tf.keras.layers.Dense(num_features)

In [ ]:
feedback_model = FeedBack(units=200, out_steps=OUT_STEPS)

TypeError: ignored

# Analytics vidhya


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import tensorflow as tf
import os
daily_df=pd.read_csv('/content/drive/MyDrive/LSTM Implementation/AHUF5/preprocessed_AHUF5.csv', low_memory=False, infer_datetime_format=True, index_col=['Timestamp'])
daily_df.head()

,hour,day,OAT,SAT,RH,CHW_Temp,Valve,RAT
Timestamp,,,,,,,,
02/05/2021 0:00,0,6,25.790000,27.3,35,28.4,100,28.0
02/05/2021 0:01,0,6,25.756667,27.3,35,28.4,100,28.0
02/05/2021 0:02,0,6,25.723333,27.3,35,28.4,100,28.0
02/05/2021 0:03,0,6,25.690000,27.3,35,28.4,100,28.0
02/05/2021 0:04,0,6,25.656667,27.3,35,28.4,100,28.0


In [ ]:
daily_df.shape

(184556, 8)

In [ ]:
train_df,test_df = daily_df[1:140000], daily_df[140000:] 

In [ ]:
train_df

,hour,day,OAT,SAT,RH,CHW_Temp,Valve,RAT
Timestamp,,,,,,,,
02/05/2021 0:01,0,6,25.756667,27.3,35,28.4,100,28.0
02/05/2021 0:02,0,6,25.723333,27.3,35,28.4,100,28.0
02/05/2021 0:03,0,6,25.690000,27.3,35,28.4,100,28.0
02/05/2021 0:04,0,6,25.656667,27.3,35,28.4,100,28.0
02/05/2021 0:05,0,6,25.623333,27.3,35,28.4,100,28.0
...,...,...,...,...,...,...,...,...
24/08/2021 21:16,21,1,31.728000,21.8,69,19.0,100,28.1
24/08/2021 21:17,21,1,31.698500,21.8,69,19.0,100,28.1
24/08/2021 21:18,21,1,31.669000,21.9,69,19.0,100,28.1


In [ ]:
test_df

,hour,day,OAT,SAT,RH,CHW_Temp,Valve,RAT
Timestamp,,,,,,,,
24/08/2021 21:21,21,1,31.580500,22.3,68,18.9,100,28.1
24/08/2021 21:22,21,1,31.551000,22.3,68,18.9,100,28.1
24/08/2021 21:23,21,1,31.521500,22.4,68,18.8,100,28.1
24/08/2021 21:24,21,1,31.492000,22.5,68,18.8,100,28.1
24/08/2021 21:25,21,1,31.462500,22.4,68,18.8,100,28.1
...,...,...,...,...,...,...,...,...
29/09/2021 16:37,16,2,33.108571,17.5,65,23.8,0,27.2
29/09/2021 16:38,16,2,33.082857,17.5,65,23.8,0,27.2
29/09/2021 16:39,16,2,33.057143,17.5,65,23.8,0,27.2


In [ ]:
train = train_df
scalers={}
for i in train_df.columns:
    scaler = MinMaxScaler(feature_range=(-1,1))
    s_s = scaler.fit_transform(train[i].values.reshape(-1,1))
    s_s=np.reshape(s_s,len(s_s))
    scalers['scaler_'+ i] = scaler
    train[i]=s_s
test = test_df
for i in train_df.columns:
    scaler = scalers['scaler_'+i]
    s_s = scaler.transform(test[i].values.reshape(-1,1))
    s_s=np.reshape(s_s,len(s_s))
    scalers['scaler_'+i] = scaler
    test[i]=s_s

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

In [ ]:
def split_series(series, n_past, n_future):
  #
  # n_past ==> no of past observations
  #
  # n_future ==> no of future observations 
  #
  X, y = list(), list()
  for window_start in range(len(series)):
    past_end = window_start + n_past
    future_end = past_end + n_future
    if future_end > len(series):
      break
    # slicing the past and future parts of the window
    past, future = series[window_start:past_end, :], series[past_end:future_end, :]
    X.append(past)
    y.append(future)
  return np.array(X), np.array(y)

In [ ]:
n_past = 120
n_future = 60 
n_features = 8

In [ ]:
X_train, y_train = split_series(train.values,n_past, n_future)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1],n_features))
y_train = y_train.reshape((y_train.shape[0], y_train.shape[1], n_features))
X_test, y_test = split_series(test.values,n_past, n_future)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1],n_features))
y_test = y_test.reshape((y_test.shape[0], y_test.shape[1], n_features))

In [ ]:

# E2D2
# n_features ==> no of features at each timestep in the data.
#
encoder_inputs = tf.keras.layers.Input(shape=(n_past, n_features))
encoder_l1 = tf.keras.layers.LSTM(100,return_sequences = True, return_state=True)
encoder_outputs1 = encoder_l1(encoder_inputs)
encoder_states1 = encoder_outputs1[1:]
encoder_l2 = tf.keras.layers.LSTM(100, return_state=True)
encoder_outputs2 = encoder_l2(encoder_outputs1[0])
encoder_states2 = encoder_outputs2[1:]
#
decoder_inputs = tf.keras.layers.RepeatVector(n_future)(encoder_outputs2[0])
#
decoder_l1 = tf.keras.layers.LSTM(100, return_sequences=True)(decoder_inputs,initial_state = encoder_states1)
decoder_l2 = tf.keras.layers.LSTM(100, return_sequences=True)(decoder_l1,initial_state = encoder_states2)
decoder_outputs2 = tf.keras.layers.Dense(n_features)(decoder_l2)
#
model_e2d2 = tf.keras.models.Model(encoder_inputs,decoder_outputs2)
#
model_e2d2.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 120, 8)]     0           []                               
                                                                                                  
 lstm (LSTM)                    [(None, 120, 100),   43600       ['input_1[0][0]']                
                                 (None, 100),                                                     
                                 (None, 100)]                                                     
                                                                                                  
 lstm_1 (LSTM)                  [(None, 100),        80400       ['lstm[0][0]']                   
                                 (None, 100),                                                 

In [ ]:
# E2D2
# n_features ==> no of features at each timestep in the data.
#
encoder_inputs = tf.keras.layers.Input(shape=(n_past, n_features))
encoder_l = tf.keras.layers.LSTM(100,return_sequences = True, return_state=True)
encoder_outputs = encoder_l(encoder_inputs)
encoder_states = encoder_outputs[1:]
encoder_l2 = tf.keras.layers.LSTM(100, return_state=True)
encoder_outputs2 = encoder_l2(encoder_outputs[0])
encoder_states2 = encoder_outputs2[1:]
#
decoder_inputs = tf.keras.layers.RepeatVector(n_future)(encoder_outputs2[0])
#
decoder_l1 = tf.keras.layers.LSTM(100, return_sequences=True)(decoder_inputs,initial_state = encoder_states)
decoder_l2 = tf.keras.layers.LSTM(100, return_sequences=True)(decoder_l1,initial_state = encoder_states2)
decoder_outputs2 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))(decoder_l2)
#
model_e1d1 = tf.keras.models.Model(encoder_inputs,decoder_outputs2)
#
model_e1d1.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 120, 8)]     0           []                               
                                                                                                  
 lstm_4 (LSTM)                  [(None, 120, 100),   43600       ['input_2[0][0]']                
                                 (None, 100),                                                     
                                 (None, 100)]                                                     
                                                                                                  
 lstm_5 (LSTM)                  [(None, 100),        80400       ['lstm_4[0][0]']                 
                                 (None, 100),                                               

In [ ]:
reduce_lr = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.90 ** x)
model_e1d1.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.Huber())
history_e1d1=model_e1d1.fit(X_train,y_train,epochs=25,validation_data=(X_test,y_test),batch_size=32,verbose=1,callbacks=[reduce_lr])
model_e2d2.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.Huber())
history_e2d2=model_e2d2.fit(X_train,y_train,epochs=25,validation_data=(X_test,y_test),batch_size=32,verbose=1,callbacks=[reduce_lr])

Epoch 1/25
4370/4370 [==============================] - 1329s 303ms/step - loss: 0.0094 - val_loss: 0.0184 - lr: 0.0010
Epoch 2/25
4370/4370 [==============================] - 1324s 303ms/step - loss: 0.0078 - val_loss: 0.0217 - lr: 9.0000e-04
Epoch 3/25
4370/4370 [==============================] - 1334s 305ms/step - loss: 0.0067 - val_loss: 0.0250 - lr: 8.1000e-04
Epoch 4/25
4370/4370 [==============================] - 1335s 306ms/step - loss: 0.0054 - val_loss: 0.0218 - lr: 7.2900e-04
Epoch 5/25
4370/4370 [==============================] - 1331s 305ms/step - loss: 0.0060 - val_loss: 0.0249 - lr: 6.5610e-04
Epoch 6/25
4370/4370 [==============================] - 1329s 304ms/step - loss: 0.0046 - val_loss: 0.0289 - lr: 5.9049e-04
Epoch 7/25
3266/4370 [=====================>........] - ETA: 5:09 - loss: 0.0039

In [ ]:
pred_e2d2=model_e2d2.predict(X_test)